In [40]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
import os
import shutil
import random
import platform

%matplotlib inline

In [4]:
summary_stats_df = pd.read_csv("../data/summary_stats.csv")

summary_stats_df["country"] = summary_stats_df["country"].apply(lambda x: "GB" if x == "UK" else x)
summary_stats_df.head()

,year,month,country,publisher,num_articles,total_words
0,2010,1,AU,3AW,1,1505
1,2010,1,AU,ABC Message Stick,2,7284
2,2010,1,AU,ABC Online,143,78050
3,2010,1,AU,ABC Regional Online,55,21602
4,2010,1,AU,Architecture and Design,14,4981


In [5]:
by_publisher = summary_stats_df.groupby(["country", "publisher"])
by_publisher = by_publisher[["num_articles", "total_words"]]
by_publisher = by_publisher.sum().reset_index().rename(columns={
    "num_articles": "publisher_tot_articles", 
    "total_words": "publisher_tot_words"
})

bins=[0, 1, 2, 3, 5, 10, 25, 100, 500, 1000, np.inf]
labels=["1", "2", "3", "4-5", "6-10", "11-25", "26-100", "101-500", "501-1000", "1001+"]

by_publisher["publisher_bin"] = pd.cut(by_publisher.publisher_tot_articles, bins=bins, labels=labels)
by_publisher.head()

,country,publisher,publisher_tot_articles,publisher_tot_words,publisher_bin
0,AU,'Bourne this way,48,14834,26-100
1,AU,10 News First,37,6240,26-100
2,AU,10 daily,221,103872,101-500
3,AU,123Jump.com,3,778,3
4,AU,2DayFM,15,3771,11-25


In [6]:
big_publishers = summary_stats_df.merge(
    by_publisher.query("publisher_bin == '1001+'"), 
    how="right", on=["country", "publisher"]
)

big_publishers.head()

,year,month,country,publisher,num_articles,total_words,publisher_tot_articles,publisher_tot_words,publisher_bin
0,2016,11,AU,9Honey,14,5941,3798,1839552,1001+
1,2016,12,AU,9Honey,52,18680,3798,1839552,1001+
2,2016,11,AU,9Honey,14,5941,3798,1839552,1001+
3,2016,12,AU,9Honey,56,20555,3798,1839552,1001+
4,2017,1,AU,9Honey,75,31747,3798,1839552,1001+


In [7]:
big_publishers

,year,month,country,publisher,num_articles,total_words,publisher_tot_articles,publisher_tot_words,publisher_bin
0,2016,11,AU,9Honey,14,5941,3798,1839552,1001+
1,2016,12,AU,9Honey,52,18680,3798,1839552,1001+
2,2016,11,AU,9Honey,14,5941,3798,1839552,1001+
3,2016,12,AU,9Honey,56,20555,3798,1839552,1001+
4,2017,1,AU,9Honey,75,31747,3798,1839552,1001+
...,...,...,...,...,...,...,...,...,...
130552,2020,5,ZA,zoutnet.co.za,156,77085,2216,1014646,1001+
130553,2020,6,ZA,zoutnet.co.za,177,87570,2216,1014646,1001+
130554,2020,7,ZA,zoutnet.co.za,195,93367,2216,1014646,1001+
130555,2020,8,ZA,zoutnet.co.za,135,66479,2216,1014646,1001+


In [8]:
CAP = 10000

def get_num_samples(x, num_samples):
    pct, max_ = x["pct"], x["total_articles"]
    n = int(np.around(num_samples * pct))
    if n > max_:
        n = max_
        
    return n

sampled_counts = []
for country, country_df in big_publishers.groupby("country"):
    
    # Get rid of month
    # For each contry group by years. Articles per year. 
    articles_by_year = country_df.groupby("year")["num_articles"].sum()
    
    # Median from the number of articles that are of the year 2015 and below. 
    num_to_sample = articles_by_year[articles_by_year.index < 2016].median()
    num_to_sample = np.min([CAP, num_to_sample])
    
    # Going through each year at a time per country. 
    for year, year_df in country_df.groupby("year"):
        publishers = year_df.groupby("publisher")["num_articles"].sum().reset_index()
        publishers["country"] = country
        publishers["year"] = year
        
        publishers["pct"] = publishers.num_articles / publishers.num_articles.sum()
        publishers["total_articles"] = publishers.num_articles
        
        publishers["sampled_articles"] = publishers.apply(
            get_num_samples, axis=1, num_samples=num_to_sample
        )
    
        sampled_counts.append(publishers[[
            "country", "year", "publisher", "total_articles", "sampled_articles"
         ]].copy())
        
big_publisher_sampling = pd.concat(sampled_counts)
big_publisher_sampling.to_csv("sampling_amounts.csv", index=False)
big_publisher_sampling.head()

,country,year,publisher,total_articles,sampled_articles
0,AU,2010,ABC Local,338,248
1,AU,2010,ABC Online,2743,2010
2,AU,2010,AdNews,7,5
3,AU,2010,Architecture and Design,147,108
4,AU,2010,Australian Personal Computer,263,193


In [9]:
print(big_publisher_sampling[["total_articles", "sampled_articles"]].sum().apply(lambda x: f"{x:,d}"))

print("\nBy Year:")
big_publisher_sampling.groupby("year")[["total_articles", "sampled_articles"]].sum()

total_articles      17,336,805
sampled_articles     1,464,959
dtype: object

By Year:


,total_articles,sampled_articles
year,,
2010,258826,84139
2011,347212,109477
2012,448073,133394
2013,510685,141804
2014,576154,142360
2015,734857,142358
2016,3006000,142357
2017,3064616,142295
2018,2575893,142342


In [10]:
big_publisher_sampling

,country,year,publisher,total_articles,sampled_articles
0,AU,2010,ABC Local,338,248
1,AU,2010,ABC Online,2743,2010
2,AU,2010,AdNews,7,5
3,AU,2010,Architecture and Design,147,108
4,AU,2010,Australian Personal Computer,263,193
...,...,...,...,...,...
60,ZA,2020,timeslive.co.za,2724,211
61,ZA,2020,weetracker.com,662,51
62,ZA,2020,wheels24.co.za,2323,180
63,ZA,2020,za.ign.com,2319,179


## Sampling

In [19]:
base_path = '/Users/leo/Documents/GitHub/peace-speech-project2/'

In [53]:
# Things to change
# 1. base_path
# 2. base_dir:  'clean' -> '?' to a folder name which has all full cleaned data. (Folder split into Country, Year, Publisher)

os_name = platform.system()

base_dir = os.path.join(base_path, 'data', 'clean') 

counter =0

for index, row in big_publisher_sampling.iterrows():
    row.publisher = slugify(row.publisher)
    current_dir = os.path.join(base_dir,row.country,row.publisher,str(row.year))
    
    if os_name == 'Windows':
        print(current_dir)
        first_part, second_part = current_dir.split('clean\\')
        
    else:
        print(current_dir)
        first_part, second_part = current_dir.split('/clean/')
    
    copy_dir = os.path.join(first_part,'clean_subsample',second_part)
    
    if os.path.isdir(current_dir):
        articles = os.listdir(current_dir)
        sample_num = min(row.sampled_articles, len(articles))
        sampled_articles = random.choices(articles, k=sample_num)

        if not os.path.exists(copy_dir):
            os.makedirs(copy_dir)

        for article in sampled_articles:
            shutil.copyfile(os.path.join(current_dir, article), os.path.join(copy_dir, article))
    
    else:
        counter +=1
        print("Not a directory:" + current_dir)

print("Counter: " + counter)

/Users/leo/Documents/GitHub/peace-speech-project2/data/clean/AU/abc-local/2010
/Users/leo/Documents/GitHub/peace-speech-project2/data/clean/AU/abc-online/2010
/Users/leo/Documents/GitHub/peace-speech-project2/data/clean/AU/adnews/2010
/Users/leo/Documents/GitHub/peace-speech-project2/data/clean/AU/architecture-and-design/2010
/Users/leo/Documents/GitHub/peace-speech-project2/data/clean/AU/australian-personal-computer/2010
/Users/leo/Documents/GitHub/peace-speech-project2/data/clean/AU/ballarat-courier/2010
/Users/leo/Documents/GitHub/peace-speech-project2/data/clean/AU/bendigo-advertiser/2010
/Users/leo/Documents/GitHub/peace-speech-project2/data/clean/AU/brisbane-times/2010
/Users/leo/Documents/GitHub/peace-speech-project2/data/clean/AU/crn-australia/2010
/Users/leo/Documents/GitHub/peace-speech-project2/data/clean/AU/caradvice/2010
/Users/leo/Documents/GitHub/peace-speech-project2/data/clean/AU/computerworld-australia/2010
/Users/leo/Documents/GitHub/peace-speech-project2/data/clean/

/Users/leo/Documents/GitHub/peace-speech-project2/data/clean/AU/bundaberg-news-mail/2012
/Users/leo/Documents/GitHub/peace-speech-project2/data/clean/AU/crn-australia/2012
/Users/leo/Documents/GitHub/peace-speech-project2/data/clean/AU/capsule-computers/2012
/Users/leo/Documents/GitHub/peace-speech-project2/data/clean/AU/caradvice/2012
/Users/leo/Documents/GitHub/peace-speech-project2/data/clean/AU/clarence-valley-daily-examiner/2012
/Users/leo/Documents/GitHub/peace-speech-project2/data/clean/AU/coffs-coast-advocate/2012
/Users/leo/Documents/GitHub/peace-speech-project2/data/clean/AU/computerworld-australia/2012
/Users/leo/Documents/GitHub/peace-speech-project2/data/clean/AU/courier-mail/2012
/Users/leo/Documents/GitHub/peace-speech-project2/data/clean/AU/crikey/2012
/Users/leo/Documents/GitHub/peace-speech-project2/data/clean/AU/e-online/2012
/Users/leo/Documents/GitHub/peace-speech-project2/data/clean/AU/eurosport-com-au/2012
/Users/leo/Documents/GitHub/peace-speech-project2/data/cl

/Users/leo/Documents/GitHub/peace-speech-project2/data/clean/AU/my-sailing/2013
/Users/leo/Documents/GitHub/peace-speech-project2/data/clean/AU/news-com-au/2013
/Users/leo/Documents/GitHub/peace-speech-project2/data/clean/AU/nrl-com/2013
/Users/leo/Documents/GitHub/peace-speech-project2/data/clean/AU/newcastle-herald/2013
/Users/leo/Documents/GitHub/peace-speech-project2/data/clean/AU/newsmaker/2013
/Users/leo/Documents/GitHub/peace-speech-project2/data/clean/AU/noise11/2013
/Users/leo/Documents/GitHub/peace-speech-project2/data/clean/AU/northern-star/2013
/Users/leo/Documents/GitHub/peace-speech-project2/data/clean/AU/on-line-opinion/2013
/Users/leo/Documents/GitHub/peace-speech-project2/data/clean/AU/pedestrian-tv/2013
/Users/leo/Documents/GitHub/peace-speech-project2/data/clean/AU/proactive-investors-australia/2013
/Users/leo/Documents/GitHub/peace-speech-project2/data/clean/AU/property-observer/2013
/Users/leo/Documents/GitHub/peace-speech-project2/data/clean/AU/radio-australia/201

/Users/leo/Documents/GitHub/peace-speech-project2/data/clean/AU/the-australian-financial-review/2014
/Users/leo/Documents/GitHub/peace-speech-project2/data/clean/AU/the-border-mail/2014
/Users/leo/Documents/GitHub/peace-speech-project2/data/clean/AU/the-cairns-post/2014
/Users/leo/Documents/GitHub/peace-speech-project2/data/clean/AU/the-canberra-times/2014
/Users/leo/Documents/GitHub/peace-speech-project2/data/clean/AU/the-conversation/2014
/Users/leo/Documents/GitHub/peace-speech-project2/data/clean/AU/the-conversation-au/2014
/Users/leo/Documents/GitHub/peace-speech-project2/data/clean/AU/the-conversation-uk/2014
/Users/leo/Documents/GitHub/peace-speech-project2/data/clean/AU/the-conversation-us/2014
/Users/leo/Documents/GitHub/peace-speech-project2/data/clean/AU/the-interpreter/2014
/Users/leo/Documents/GitHub/peace-speech-project2/data/clean/AU/the-new-daily/2014
/Users/leo/Documents/GitHub/peace-speech-project2/data/clean/AU/the-queensland-times/2014
/Users/leo/Documents/GitHub/pe

/Users/leo/Documents/GitHub/peace-speech-project2/data/clean/AU/the-australian-financial-review/2015
/Users/leo/Documents/GitHub/peace-speech-project2/data/clean/AU/the-border-mail/2015
/Users/leo/Documents/GitHub/peace-speech-project2/data/clean/AU/the-cairns-post/2015
/Users/leo/Documents/GitHub/peace-speech-project2/data/clean/AU/the-canberra-times/2015
/Users/leo/Documents/GitHub/peace-speech-project2/data/clean/AU/the-conversation-au/2015
/Users/leo/Documents/GitHub/peace-speech-project2/data/clean/AU/the-conversation-uk/2015
/Users/leo/Documents/GitHub/peace-speech-project2/data/clean/AU/the-conversation-us/2015
/Users/leo/Documents/GitHub/peace-speech-project2/data/clean/AU/the-interpreter/2015
/Users/leo/Documents/GitHub/peace-speech-project2/data/clean/AU/the-new-daily/2015
/Users/leo/Documents/GitHub/peace-speech-project2/data/clean/AU/the-queensland-times/2015
/Users/leo/Documents/GitHub/peace-speech-project2/data/clean/AU/the-roar/2015
/Users/leo/Documents/GitHub/peace-spee

...

/Users/leo/Documents/GitHub/peace-speech-project2/data/clean/ZA/north-coast-courier/2017
/Users/leo/Documents/GitHub/peace-speech-project2/data/clean/ZA/northglen-news/2017
/Users/leo/Documents/GitHub/peace-speech-project2/data/clean/ZA/political-analysis-south-africa/2017
/Users/leo/Documents/GitHub/peace-speech-project2/data/clean/ZA/politicsweb/2017
/Users/leo/Documents/GitHub/peace-speech-project2/data/clean/ZA/premier-soccer-league/2017
/Users/leo/Documents/GitHub/peace-speech-project2/data/clean/ZA/pretoria-east-rekord/2017
/Users/leo/Documents/GitHub/peace-speech-project2/data/clean/ZA/rnews/2017
/Users/leo/Documents/GitHub/peace-speech-project2/data/clean/ZA/randburg-sun/2017
/Users/leo/Documents/GitHub/peace-speech-project2/data/clean/ZA/randfontein-herald/2017
/Users/leo/Documents/GitHub/peace-speech-project2/data/clean/ZA/reuters-africa/2017
/Users/leo/Documents/GitHub/peace-speech-project2/data/clean/ZA/rising-sun-chatsworth/2017
/Users/leo/Documents/GitHub/peace-speech-pro

/Users/leo/Documents/GitHub/peace-speech-project2/data/clean/ZA/marklives-com/2018
/Users/leo/Documents/GitHub/peace-speech-project2/data/clean/ZA/media-update/2018
/Users/leo/Documents/GitHub/peace-speech-project2/data/clean/ZA/memeburn/2018
/Users/leo/Documents/GitHub/peace-speech-project2/data/clean/ZA/mining-mx/2018
/Users/leo/Documents/GitHub/peace-speech-project2/data/clean/ZA/mining-review/2018
/Users/leo/Documents/GitHub/peace-speech-project2/data/clean/ZA/mining-technology/2018
/Users/leo/Documents/GitHub/peace-speech-project2/data/clean/ZA/moneyweb-co-za/2018
/Users/leo/Documents/GitHub/peace-speech-project2/data/clean/ZA/mpumalanga-news/2018
/Users/leo/Documents/GitHub/peace-speech-project2/data/clean/ZA/mybroadband/2018
/Users/leo/Documents/GitHub/peace-speech-project2/data/clean/ZA/mygaming/2018
/Users/leo/Documents/GitHub/peace-speech-project2/data/clean/ZA/news24/2018
/Users/leo/Documents/GitHub/peace-speech-project2/data/clean/ZA/north-coast-courier/2018
/Users/leo/Docu

/Users/leo/Documents/GitHub/peace-speech-project2/data/clean/ZA/highway-mail/2019
/Users/leo/Documents/GitHub/peace-speech-project2/data/clean/ZA/ign-africa/2019
/Users/leo/Documents/GitHub/peace-speech-project2/data/clean/ZA/imiesa/2019
/Users/leo/Documents/GitHub/peace-speech-project2/data/clean/ZA/it-news-africa/2019
/Users/leo/Documents/GitHub/peace-speech-project2/data/clean/ZA/it-online/2019
/Users/leo/Documents/GitHub/peace-speech-project2/data/clean/ZA/itweb/2019
/Users/leo/Documents/GitHub/peace-speech-project2/data/clean/ZA/independent-online/2019
/Users/leo/Documents/GitHub/peace-speech-project2/data/clean/ZA/kickoff-com/2019
/Users/leo/Documents/GitHub/peace-speech-project2/data/clean/ZA/krugersdorp-news/2019
/Users/leo/Documents/GitHub/peace-speech-project2/data/clean/ZA/lowvelder/2019
/Users/leo/Documents/GitHub/peace-speech-project2/data/clean/ZA/mail-and-guardian/2019
/Users/leo/Documents/GitHub/peace-speech-project2/data/clean/ZA/malawi24/2019
/Users/leo/Documents/GitH

/Users/leo/Documents/GitHub/peace-speech-project2/data/clean/ZA/southcoastherald-co-za/2019
/Users/leo/Documents/GitHub/peace-speech-project2/data/clean/ZA/sport24-co-za/2019
/Users/leo/Documents/GitHub/peace-speech-project2/data/clean/ZA/talkofthetown-co-za/2019
/Users/leo/Documents/GitHub/peace-speech-project2/data/clean/ZA/techcentral-co-za/2019
/Users/leo/Documents/GitHub/peace-speech-project2/data/clean/ZA/themediaonline-co-za/2019
/Users/leo/Documents/GitHub/peace-speech-project2/data/clean/ZA/thesouthafrican-com/2019
/Users/leo/Documents/GitHub/peace-speech-project2/data/clean/ZA/timeslive-co-za/2019
/Users/leo/Documents/GitHub/peace-speech-project2/data/clean/ZA/weetracker-com/2019
/Users/leo/Documents/GitHub/peace-speech-project2/data/clean/ZA/wheels24-co-za/2019
/Users/leo/Documents/GitHub/peace-speech-project2/data/clean/ZA/za-ign-com/2019
/Users/leo/Documents/GitHub/peace-speech-project2/data/clean/ZA/zoutnet-co-za/2019
/Users/leo/Documents/GitHub/peace-speech-project2/data

TypeError: can only concatenate str (not "int") to str

### Number of paths where the directory wasn't found.

In [54]:
counter

1350

#### Two reasons for this. 
#### 1. The folder wasn't created beacuse none of the files in the folder were selected when subsampled. <br> (i.e. our 'clean' folder is output of running Clean Text Files- SAMPLE.ipynb, with MAX_SIZE = 50). <br> For example, none of the articles from year 2010 of the publisher skysports-com were selected thus the folder wasn't created. This won't be a problem on your side since you are using all the data. 
#### 2. Most of the US publishers don't have a 2019/2020 year folder. I remember you saying that there was a problem in how some of the US raw files were formatted. I'm not sure if you solved this problem but I think this is the reason why there are so many US 2019/2020 directories not found. 